In [1]:
!pip install pyspark
!wget https://datahub.io/core/population/r/population.json -O population.json -q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 60.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=59d6eb6bb0e21c6604a25fe6f1ec38c9acab46215bda2a1159ea179fe32d7d5a
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, Window as W
from pyspark.sql.functions import lag, col, coalesce, lit, avg
from pyspark.sql.types import DoubleType

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
df = spark.read.json("/content/population.json")

##Задача
Модернизируйте заготовку заменив все участки ```<put your code here>``` на ваш код для того, что бы:
* вычислить прирост населения в каждой стране от года к году (воспользуйтесь функцией [lag](https://spark.apache.org/docs/2.3.0/api/sql/#lag) или [lead](https://spark.apache.org/docs/2.3.0/api/sql/#lead))
* вычислить средний прирост за период с 1990 по 2018 год
* вычислить и вывести на экран страны с трендом на убыль населения с 1990 по 2018 годы

In [ ]:
df = df.withColumnRenamed("Country Code","Country_Code") \
  .withColumnRenamed("Country Name","Country_Name")
  
df.createOrReplaceTempView('population')

In [ ]:
df.show()

+------------+------------+------------+----+
|Country_Code|Country_Name|       Value|Year|
+------------+------------+------------+----+
|         ARB|  Arab World| 9.2197753E7|1960|
|         ARB|  Arab World|  9.472451E7|1961|
|         ARB|  Arab World| 9.7334442E7|1962|
|         ARB|  Arab World|1.00034179E8|1963|
|         ARB|  Arab World| 1.0283276E8|1964|
|         ARB|  Arab World|1.05736431E8|1965|
|         ARB|  Arab World| 1.0875861E8|1966|
|         ARB|  Arab World|1.11899364E8|1967|
|         ARB|  Arab World|1.15136178E8|1968|
|         ARB|  Arab World|1.18437195E8|1969|
|         ARB|  Arab World| 1.2178565E8|1970|
|         ARB|  Arab World|1.25164745E8|1971|
|         ARB|  Arab World|1.28598734E8|1972|
|         ARB|  Arab World|1.32161298E8|1973|
|         ARB|  Arab World|1.35952252E8|1974|
|         ARB|  Arab World|1.40040649E8|1975|
|         ARB|  Arab World|1.44453278E8|1976|
|         ARB|  Arab World|1.49161891E8|1977|
|         ARB|  Arab World|1.54111

##1) Прирост населедния в странах от года к году

SQL запрос:

In [ ]:
incr = spark.sql('''
select 
  Country_Name, Year, (
  Value - lag(Value, 1, 0) over (
    partition by Country_Name 
    order by Value
  )) as Increase 
  from population''')

incr.createOrReplaceTempView('increasing')
incr.show(10)

+------------+----+---------+
|Country_Name|Year| Increase|
+------------+----+---------+
| Afghanistan|1960|8996973.0|
| Afghanistan|1961| 172437.0|
| Afghanistan|1962| 182031.0|
| Afghanistan|1963| 191764.0|
| Afghanistan|1964| 201576.0|
| Afghanistan|1965| 211539.0|
| Afghanistan|1966| 218516.0|
| Afghanistan|1967| 225090.0|
| Afghanistan|1968| 237137.0|
| Afghanistan|1969| 256713.0|
+------------+----+---------+
only showing top 10 rows



Манипуляции с DF:

In [ ]:
partition = W.partitionBy('Country_Name').orderBy('Year')
incrDFAPI = df.select('Country_Name', 'Year', (col('Value') - lag(col('Value'), 1, 0).over(partition)).alias('Increase'))
incrDFAPI.show(10)

+------------+----+---------+
|Country_Name|Year| Increase|
+------------+----+---------+
| Afghanistan|1960|8996973.0|
| Afghanistan|1961| 172437.0|
| Afghanistan|1962| 182031.0|
| Afghanistan|1963| 191764.0|
| Afghanistan|1964| 201576.0|
| Afghanistan|1965| 211539.0|
| Afghanistan|1966| 218516.0|
| Afghanistan|1967| 225090.0|
| Afghanistan|1968| 237137.0|
| Afghanistan|1969| 256713.0|
+------------+----+---------+
only showing top 10 rows



##2) Вычислить средний прирост за период с 1990 по 2018 год

SQL-запрос:

In [ ]:
avgIncr = spark.sql('''
  select Country_Name, avg(Increase) as Average_Increase
    from increasing 
    where Year between 1990 and 2018 
    group by Country_Name 
    order by Average_Increase desc
''')
avgIncr.show(10)

+--------------------+--------------------+
|        Country_Name|    Average_Increase|
+--------------------+--------------------+
|               World|  8.28725908275862E7|
| Low & middle income| 7.549617962068966E7|
|    IDA & IBRD total| 7.537248937931034E7|
|       Middle income|6.2316960172413796E7|
|           IBRD only|4.6972815172413796E7|
|Early-demographic...|  4.36805435862069E7|
| Lower middle income| 4.103903596551724E7|
|           IDA total| 2.839967420689655E7|
|          South Asia| 2.433401175862069E7|
|South Asia (IDA &...| 2.433401175862069E7|
+--------------------+--------------------+
only showing top 10 rows



In [ ]:
spark.catalog.listTables()

[Table(name='increasing', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='population', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

##3) Вычислить и вывести на экран страны с трендом на убыль населения с 1990 по 2018 годы

Манипуляции с DF:

In [ ]:
decreaseTrend = df.where(col('Year').between(1990, 2018)) \
  .withColumn("cnt_country", col('Value') - lag("Value").over(partition)) \
  .groupBy("Country_Name") \
  .agg(avg("cnt_country").alias("Trend")) \
  .where("Trend < 0")

decreaseTrend.show(10)

+--------------------+-------------------+
|        Country_Name|              Trend|
+--------------------+-------------------+
|             Albania| -15005.92857142857|
|             Armenia|-20942.678571428572|
|             Belarus|-25196.464285714286|
|Bosnia and Herzeg...| -40696.21428571428|
|            Bulgaria| -60473.28571428572|
|Central Europe an...|-293971.64285714284|
|             Croatia| -24625.89285714286|
|             Estonia| -8828.464285714286|
|             Georgia| -38408.96428571428|
|             Hungary|-21372.285714285714|
+--------------------+-------------------+
only showing top 10 rows

